In [ ]:

!pip -q install -U "huggingface_hub<1.0,>=0.34.0" accelerate safetensors opencv-python
!pip -q install -U diffusers transformers


In [ ]:

from google.colab import drive
drive.mount("/content/drive")

%cd "/content/drive/Othercomputers/ה-Mac שלי/StormVision"
!pwd
!ls


In [ ]:

import os, json, random
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import torch
from PIL import Image, ImageDraw, ImageFilter, ImageEnhance
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from huggingface_hub import login
from diffusers import ControlNetModel, StableDiffusionControlNetInpaintPipeline, DPMSolverMultistepScheduler

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32
print("device:", device, "| torch:", torch.__version__)

BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data"

IMG_TRAIN_DIR = DATA_DIR / "images" / "train"
IMG_VAL_DIR   = DATA_DIR / "images" / "val"
ANN_DIR       = DATA_DIR / "annotations"

TRAIN_JSON    = ANN_DIR / "instances_train.json"
VAL_JSON      = ANN_DIR / "instances_val.json"

assert IMG_TRAIN_DIR.exists(), f"Missing: {IMG_TRAIN_DIR}"
assert IMG_VAL_DIR.exists(),   f"Missing: {IMG_VAL_DIR}"
assert TRAIN_JSON.exists(),    f"Missing: {TRAIN_JSON}"
assert VAL_JSON.exists(),      f"Missing: {VAL_JSON}"

# Output: ONE folder only
OUT_DIR = BASE_DIR / "storm_synth_out_onefolder"
OUT_IMAGES_DIR = OUT_DIR / "images"
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_IMAGES_DIR.mkdir(parents=True, exist_ok=True)
print("OK. Output:", OUT_DIR)


In [ ]:

tok = os.environ.get("HF_TOKEN", None)
if tok:
    login(token=tok)
else:
    login()

##  Load COCO + build image-level dataframe (train/val pool)
- Loads COCO JSONs, infers person category IDs, builds image-level metadata (bboxes/counts), and merges train+val into one pool for generation.


In [ ]:
#  Load a COCO JSON file and return the raw dict plus images/annotations/categories as DataFrames.
def load_coco(json_path: Path):
    with open(json_path, "r") as f:
        coco = json.load(f)
    images_df = pd.DataFrame(coco.get("images", []))         # COCO images table
    ann_df    = pd.DataFrame(coco.get("annotations", []))    # COCO annotations table
    cats_df   = pd.DataFrame(coco.get("categories", []))     # COCO categories table
    return coco, images_df, ann_df, cats_df

#  Infer the category IDs that correspond to "person-like" classes from the COCO categories table.
def infer_person_category_ids(cats_df: pd.DataFrame, target_names=("person","swimmer","human")):
    if cats_df is None or cats_df.empty:
        return set()
    if not {"id","name"}.issubset(cats_df.columns):
        return set()
    names = cats_df["name"].astype(str).str.lower()
    hits = cats_df[names.isin(set(target_names))]
    if not hits.empty:
        return set(hits["id"].astype(int).tolist())
    hits2 = cats_df[names.str.contains("person|human|swim", regex=True, na=False)] # exact match first
    return set(hits2["id"].astype(int).tolist())

#  Build an image-level DataFrame with per-image bbox lists and person presence/count signals.
def build_image_level_df(images_df: pd.DataFrame, ann_df: pd.DataFrame, person_cat_ids: set):
    images_df = images_df.copy()
    if ann_df.empty:
        ann_df = pd.DataFrame(columns=["image_id","category_id","bbox"])

    if {"image_id","category_id","bbox"}.issubset(ann_df.columns) and len(person_cat_ids) > 0:
        person_ann = ann_df[ann_df["category_id"].isin(person_cat_ids)].copy()
    else:
        person_ann = pd.DataFrame(columns=["image_id","category_id","bbox"])

    boxes_per_image = (
        person_ann.groupby("image_id")["bbox"]
        .apply(list).reset_index().rename(columns={"bbox":"bboxes"})
    )

    df = images_df.merge(boxes_per_image, left_on="id", right_on="image_id", how="left")
    df["bboxes"] = df["bboxes"].apply(lambda x: x if isinstance(x, list) else [])
    df["has_person"] = (df["bboxes"].apply(len) > 0).astype(int)
    df["persons_count"] = df["bboxes"].apply(len)
    df["aspect_ratio"] = df["width"] / df["height"]
    df = df.drop(columns=["image_id"], errors="ignore")
    return df[["id","file_name","width","height","has_person","persons_count","bboxes","aspect_ratio"]]

coco_tr, img_tr_df, ann_tr_df, cats_tr_df = load_coco(TRAIN_JSON)
coco_va, img_va_df, ann_va_df, cats_va_df = load_coco(VAL_JSON)

person_ids_tr = infer_person_category_ids(cats_tr_df)
person_ids_va = infer_person_category_ids(cats_va_df)
assert len(person_ids_tr) > 0 and len(person_ids_va) > 0, "Could not infer person category ids"

train_image_df = build_image_level_df(img_tr_df, ann_tr_df, person_ids_tr)
val_image_df   = build_image_level_df(img_va_df, ann_va_df, person_ids_va)

print("train:", train_image_df.shape, "pos%:", round(train_image_df["has_person"].mean()*100, 2))
print("val:",   val_image_df.shape,   "pos%:", round(val_image_df["has_person"].mean()*100, 2))

# Keep only person categories (from TRAIN for consistency)
person_cats_tr = cats_tr_df[cats_tr_df["id"].isin(list(person_ids_tr))].copy()
person_cats_tr = person_cats_tr[["id","name"]].to_dict(orient="records")
PERSON_CAT_ID  = int(person_cats_tr[0]["id"])

# Merge train+val into one pool (no split in output)
train_pool = train_image_df.copy()
train_pool["src_split"] = "train"
train_pool["src_dir"] = str(IMG_TRAIN_DIR)

val_pool = val_image_df.copy()
val_pool["src_split"] = "val"
val_pool["src_dir"] = str(IMG_VAL_DIR)

full_pool = pd.concat([train_pool, val_pool], ignore_index=True)
print("full_pool:", full_pool.shape, "pos%:", round(full_pool["has_person"].mean()*100, 2))


##  Scenario prompts (weather/visibility conditions)
- Defines prompt templates, sampling weights, and a negative prompt to reduce layout changes and artifacts.


In [ ]:


PROMPT_PREFIX = (
    "aerial drone photo over the sea, same camera angle, same composition, "
    "preserve existing boats/buoys/people positions and sizes, realistic"
)

SCENARIOS = {
    "haze": [
        f"{PROMPT_PREFIX}, heavy haze, reduced contrast, washed-out colors, distant horizon barely visible",
    ],
    "fog": [
        f"{PROMPT_PREFIX}, dense sea fog layer, very low visibility, soft diffuse light, muted colors",
        f"{PROMPT_PREFIX}, patchy fog banks, uneven visibility, some areas clearer, some obscured",
    ],
    "poor_visibility": [
        f"{PROMPT_PREFIX}, strong atmospheric scattering, low contrast, horizon fades, visibility strongly reduced",
    ],
    "storm_rain": [
        f"{PROMPT_PREFIX}, heavy rain storm, rain streaks, rain curtain, wind-driven spray, dark overcast sky",
    ],
    "wind_spray": [
        f"{PROMPT_PREFIX}, strong wind, sea spray, spindrift, streaked surface texture, overcast lighting",
    ],
    "high_sea_state": [
        f"{PROMPT_PREFIX}, rough sea, higher waves, whitecaps, foam lines, choppy surface",
    ],
    "extreme_waves": [
        f"{PROMPT_PREFIX}, extremely rough sea, large swell, chaotic waves, lots of whitecaps, heavy foam patterns",
    ],
    "whiteout_mix": [
        f"{PROMPT_PREFIX}, rain plus fog mix, extremely low visibility but objects still faintly visible, soft edges",
    ],
    "low_light_storm": [
        f"{PROMPT_PREFIX}, low-light storm, dim ambient light, bluish-gray tone, low saturation, heavy clouds",
    ],
}

SCENARIO_WEIGHTS = {
    "haze": 1.2,
    "fog": 1.2,
    "poor_visibility": 1.2,
    "storm_rain": 1.3,
    "wind_spray": 1.1,
    "high_sea_state": 1.2,
    "extreme_waves": 1.2,
    "whiteout_mix": 0.6,
    "low_light_storm": 0.8,
}

NEG_PROMPT = (
    "object removal, missing boats, missing people, extra boats, extra people, new large objects, hallucinated ships, "
    "changed composition, changed camera angle, crop, zoom, perspective change, "
    "text, watermark, logo, UI elements, black squares, checkerboard, artifacts, "
    "cartoon, illustration, CGI, lowres, blurry, deformed"
)

#  Randomly sample a scenario (weighted) and return its name plus a matching text prompt.
def sample_scenario_and_prompt(rng: random.Random):
    keys = list(SCENARIOS.keys())
    weights = [SCENARIO_WEIGHTS.get(k, 1.0) for k in keys]
    scenario = rng.choices(keys, weights=weights, k=1)[0]
    prompt = rng.choice(SCENARIOS[scenario])
    return scenario, prompt


##  Inpainting masks + ControlNet conditioning (Canny)
- Builds masks that protect people and edges while allowing controlled edits in sky/sea bands.
- Builds a Canny edge image used as ControlNet input.


In [ ]:
#  Detect a large mostly-black rectangular region to protect it from inpainting.
def detect_large_black_boxes(img_rgb: Image.Image, thr=18, min_area=1500):
    arr = np.array(img_rgb)
    black = (arr[...,0] < thr) & (arr[...,1] < thr) & (arr[...,2] < thr)
    ys, xs = np.where(black)
    if len(xs) == 0:
        return None
    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    if (x1 - x0) * (y1 - y0) < min_area:
        return None
    return (int(x0), int(y0), int(x1), int(y1))


#  Create an inpaint mask that repaints sky and optionally sea while protecting persons and edges.
def build_weather_inpaint_mask(
    img_rgb: Image.Image,
    bboxes,
    scenario: str,
    keep_pad=140,
    sky_frac=0.32,
    sea_from_frac=0.55,
    keep_edges=True,
    edge_canny1=140,
    edge_canny2=260,
    edge_dilate=5,
    keep_black_boxes=True,
):
    """
    Returns mask L:
    white (255) = repaint (inpaint)
    black (0)   = keep from original

    Strategy:
    - repaint mainly SKY band always (minimal structure changes)
    - for wave scenarios, also repaint SEA lower band
    - always protect persons via bbox rectangles (black)
    - optionally protect edges globally (black where edges exist)
    """
    W, H = img_rgb.size
    mask = Image.new("L", (W, H), 0)
    mask_np = np.array(mask)

    # 1) Allow repaint on sky band
    y_sky = int(np.clip(sky_frac, 0.05, 0.60) * H)
    mask_np[0:y_sky, :] = 255

    # 2) Allow repaint on sea band only for wave-related scenarios
    if scenario in ["high_sea_state", "extreme_waves", "wind_spray", "storm_rain", "poor_visibility", "fog"]:
        y0 = int(np.clip(sea_from_frac, 0.30, 0.90) * H)
        mask_np[y0:H, :] = 255

    # 3) Protect objects (persons) with padding
    for (x, y, w, h) in bboxes:
        x0 = max(0, int(x - keep_pad)); y0 = max(0, int(y - keep_pad))
        x1 = min(W, int(x + w + keep_pad)); y1 = min(H, int(y + h + keep_pad))
        mask_np[y0:y1, x0:x1] = 0

    # 4) Protect global edges to keep geometry stable
    if keep_edges:
        arr = np.array(img_rgb)
        gray = cv2.cvtColor(arr, cv2.COLOR_RGB2GRAY)
        edges = cv2.Canny(gray, edge_canny1, edge_canny2)
        k = edge_dilate if edge_dilate % 2 == 1 else edge_dilate + 1
        kernel = np.ones((k, k), np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=1)
        mask_np[edges > 0] = 0

    # 5) Protect black boxes if exist
    if keep_black_boxes:
        bb = detect_large_black_boxes(img_rgb)
        if bb:
            x0,y0,x1,y1 = bb
            mask_np[y0:y1, x0:x1] = 0

    return Image.fromarray(mask_np.astype(np.uint8), mode="L")


#  Build a 3-channel Canny edge image to use as the ControlNet conditioning input.
def build_canny_control_image(img_rgb: Image.Image, canny1=100, canny2=200):
    arr = np.array(img_rgb)
    gray = cv2.cvtColor(arr, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, canny1, canny2)
    edges_3 = np.stack([edges, edges, edges], axis=-1)
    return Image.fromarray(edges_3)

##  post-processing overlays (fog/rain/low light)
- Adds lightweight visual effects after generation to strengthen the target weather condition without changing geometry.


In [ ]:
#  Apply a simple haze/fog look by blending a tinted overlay and reducing contrast/saturation.
def add_fog_haze(img: Image.Image, strength=0.35, tint=(200, 200, 200)):
    overlay = Image.new("RGB", img.size, tint)
    out = Image.blend(img, overlay, alpha=float(np.clip(strength, 0, 0.85)))
    out = ImageEnhance.Contrast(out).enhance(1.0 - 0.25*strength)
    out = ImageEnhance.Color(out).enhance(1.0 - 0.30*strength)
    return out

#  Add synthetic rain streaks using random slanted line drawing and compositing.
def add_rain_streaks(img: Image.Image, amount=0.35, thickness=1, slant=12):
    arr = np.array(img).astype(np.uint8)
    h, w = arr.shape[:2]
    rain = np.zeros((h, w), dtype=np.uint8)

    n = int((h*w) * (0.00006 + 0.00025*amount))
    rng = np.random.default_rng(1234)
    xs = rng.integers(0, w, size=n)
    ys = rng.integers(0, h, size=n)
    for x, y in zip(xs, ys):
        x2 = int(x + slant)
        y2 = int(y + 22 + 40*amount)
        cv2.line(rain, (x, y), (x2, y2), 255, thickness=thickness)

    rain = cv2.GaussianBlur(rain, (3,3), 0)
    rain_rgb = np.stack([rain, rain, rain], axis=-1)

    out = cv2.addWeighted(arr, 1.0, rain_rgb, 0.22 + 0.35*amount, 0)
    return Image.fromarray(out)

#  Darken and desaturate an image to simulate low-light storm conditions.
def add_low_light(img: Image.Image, amount=0.35):
    out = ImageEnhance.Brightness(img).enhance(1.0 - 0.55*amount)
    out = ImageEnhance.Color(out).enhance(1.0 - 0.45*amount)
    return out

#  Apply a scenario-specific post overlay (fog/rain/low-light) to strengthen the weather effect.
def apply_weather_overlay(img: Image.Image, scenario: str):
    if scenario in ["haze"]:
        img = add_fog_haze(img, strength=0.40, tint=(205,205,205))
    elif scenario in ["fog", "poor_visibility"]:
        img = add_fog_haze(img, strength=0.55, tint=(210,210,210))
        img = img.filter(ImageFilter.GaussianBlur(radius=0.8))
    elif scenario in ["whiteout_mix"]:
        img = add_fog_haze(img, strength=0.70, tint=(220,220,220))
        img = add_rain_streaks(img, amount=0.55, thickness=1, slant=8)
        img = img.filter(ImageFilter.GaussianBlur(radius=1.1))
    elif scenario in ["storm_rain"]:
        img = add_rain_streaks(img, amount=0.55, thickness=1, slant=10)
        img = add_fog_haze(img, strength=0.25, tint=(190,190,190))
    elif scenario in ["wind_spray"]:
        img = add_fog_haze(img, strength=0.28, tint=(200,200,200))
    elif scenario in ["low_light_storm"]:
        img = add_low_light(img, amount=0.55)
        img = add_fog_haze(img, strength=0.25, tint=(180,180,190))
    elif scenario in ["high_sea_state", "extreme_waves"]:
        img = add_fog_haze(img, strength=0.18, tint=(195,195,195))
    return img



## Load Stable Diffusion Inpainting + ControlNet (Canny)
- Loads the inpainting model and matching ControlNet, configures scheduler and memory optimizations.


In [ ]:

MODEL_INPAINT_ID    = "runwayml/stable-diffusion-inpainting"
CONTROLNET_CANNY_ID = "lllyasviel/control_v11p_sd15_canny"

controlnet = ControlNetModel.from_pretrained(CONTROLNET_CANNY_ID, torch_dtype=dtype)

pipe = StableDiffusionControlNetInpaintPipeline.from_pretrained(
    MODEL_INPAINT_ID,
    controlnet=controlnet,
    torch_dtype=dtype,
    safety_checker=None,
    requires_safety_checker=False,
).to(device)

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_attention_slicing()
pipe.enable_vae_slicing()
print("Pipes loaded.")

## Core generation function (ControlNet Inpaint + optional overlay)
- Generates one synthetic image per input using: sampled scenario prompt + inpaint mask + Canny control image.


In [ ]:

#  Generate a single weather-augmented image using ControlNet inpainting while preserving people via masks.
@torch.inference_mode()
def gen_controlnet_weather(
    img_path: Path,
    bboxes,
    seed: int,
    max_side=768,
    steps=26,
    guidance_scale=4.0,
    controlnet_scale=1.10,
    keep_pad=160,
    sky_frac=0.32,
    sea_from_frac=0.60,
    keep_edges=True,
    edge_dilate=5,
    apply_overlay=True,
):
    img = Image.open(img_path).convert("RGB")
    img_small, orig_size, (sx, sy) = resize_to_max_side(img, max_side=max_side)
    b_scaled = scale_bboxes(bboxes, sx, sy)

    rng = random.Random(seed)
    scenario, prompt = sample_scenario_and_prompt(rng)

    mask = build_weather_inpaint_mask(
        img_small,
        b_scaled,
        scenario=scenario,
        keep_pad=keep_pad,
        sky_frac=sky_frac,
        sea_from_frac=sea_from_frac,
        keep_edges=keep_edges,
        edge_dilate=edge_dilate,
        keep_black_boxes=True,
    )

    control_image = build_canny_control_image(img_small, canny1=100, canny2=200)
    g = torch.Generator(device=device).manual_seed(seed)

    out = pipe(
        prompt=prompt,
        negative_prompt=NEG_PROMPT,
        image=img_small,
        mask_image=mask,
        control_image=control_image,
        num_inference_steps=int(steps),
        guidance_scale=float(guidance_scale),
        controlnet_conditioning_scale=float(controlnet_scale),
        generator=g,
    ).images[0]

    if apply_overlay:
        out = apply_weather_overlay(out, scenario)

    return out, img_small, mask, control_image, b_scaled, scenario, prompt

## COCO writer + ONEFOLDER dataset generator
- Writes COCO structures for generated images and annotations.
- Generates positive and negative synthetic samples and logs a metadata CSV mapping original to synthetic.


In [ ]:

#  Initialize a COCO-format dict with person categories and empty images/annotations lists.
def coco_init(person_categories):
    return {"info": {}, "licenses": [], "images": [], "annotations": [], "categories": person_categories}

#  Append an image record to the COCO dict.
def add_coco_image(coco, img_id, file_name, width, height):
    coco["images"].append({"id": int(img_id), "file_name": str(file_name), "width": int(width), "height": int(height)})

#  Append a bbox annotation record to the COCO dict.
def add_coco_ann(coco, ann_id, img_id, cat_id, bbox):
    x,y,w,h = bbox
    area = float(max(0.0, w) * max(0.0, h))
    coco["annotations"].append({
        "id": int(ann_id),
        "image_id": int(img_id),
        "category_id": int(cat_id),
        "bbox": [float(x), float(y), float(w), float(h)],
        "area": area,
        "iscrowd": 0
    })

#  Sample positives/negatives, generate synthetic images, and return COCO annotations plus a metadata DataFrame.
def generate_onefolder(
    pool_df: pd.DataFrame,
    out_images_dir: Path,
    n_pos=200,
    n_neg=200,
    start_seed=3000,
    max_side=768,
    steps=26,
    guidance_scale=4.0,
    controlnet_scale=1.10,
    keep_pad=160,
    sky_frac=0.32,
    sea_from_frac=0.60,
    keep_edges=True,
    edge_dilate=5,
    apply_overlay=True,
):
    coco = coco_init(person_cats_tr)
    meta_rows = []

    pos_df = pool_df[pool_df["has_person"] == 1]
    neg_df = pool_df[pool_df["has_person"] == 0]

    pos_sample = pos_df.sample(min(n_pos, len(pos_df)), random_state=RANDOM_SEED).reset_index(drop=True)
    neg_sample = neg_df.sample(min(n_neg, len(neg_df)), random_state=RANDOM_SEED).reset_index(drop=True)

    next_img_id = 1
    next_ann_id = 1

    # POS
    for i, r in tqdm(list(enumerate(pos_sample.itertuples(index=False), start=1)), desc="ONEFOLDER POS"):
        src_dir = Path(r.src_dir)
        img_path = src_dir / r.file_name

        out, inp, mask, canny_img, b_scaled, scenario, prompt = gen_controlnet_weather(
            img_path=img_path,
            bboxes=r.bboxes,
            seed=start_seed + i,
            max_side=max_side,
            steps=steps,
            guidance_scale=guidance_scale,
            controlnet_scale=controlnet_scale,
            keep_pad=keep_pad,
            sky_frac=sky_frac,
            sea_from_frac=sea_from_frac,
            keep_edges=keep_edges,
            edge_dilate=edge_dilate,
            apply_overlay=apply_overlay,
        )

        out_name = f"synth_pos_{r.id}_{i}.jpg"
        out.save(out_images_dir / out_name, quality=95)

        W, H = out.size
        add_coco_image(coco, next_img_id, out_name, W, H)
        for bb in b_scaled:
            add_coco_ann(coco, next_ann_id, next_img_id, PERSON_CAT_ID, bb)
            next_ann_id += 1

        meta_rows.append({
            "label": 1,
            "src_split": r.src_split,
            "orig_file": r.file_name,
            "out_file": out_name,
            "scenario": scenario,
            "prompt": prompt
        })
        next_img_id += 1

    # NEG
    for i, r in tqdm(list(enumerate(neg_sample.itertuples(index=False), start=1)), desc="ONEFOLDER NEG"):
        src_dir = Path(r.src_dir)
        img_path = src_dir / r.file_name

        out, inp, mask, canny_img, b_scaled, scenario, prompt = gen_controlnet_weather(
            img_path=img_path,
            bboxes=[],
            seed=start_seed + 5000 + i,
            max_side=max_side,
            steps=steps,
            guidance_scale=guidance_scale,
            controlnet_scale=controlnet_scale,
            keep_pad=keep_pad,
            sky_frac=sky_frac,
            sea_from_frac=sea_from_frac,
            keep_edges=keep_edges,
            edge_dilate=edge_dilate,
            apply_overlay=apply_overlay,
        )

        out_name = f"synth_neg_{r.id}_{i}.jpg"
        out.save(out_images_dir / out_name, quality=95)

        W, H = out.size
        add_coco_image(coco, next_img_id, out_name, W, H)

        meta_rows.append({
            "label": 0,
            "src_split": r.src_split,
            "orig_file": r.file_name,
            "out_file": out_name,
            "scenario": scenario,
            "prompt": prompt
        })
        next_img_id += 1

    meta_df = pd.DataFrame(meta_rows)
    return coco, meta_df


# Run configuration + dataset generation + saving outputs
# Sets generation hyperparameters, runs synthesis, and saves COCO JSON + metadata CSV to the output folder.



CFG = {
    "max_side": 768,
    "steps": 35,
    "guidance_scale": 7.5,
    "controlnet_scale": 0.85,
    "keep_pad": 220,
    "sky_frac": 0.48,
    "sea_from_frac": 0.45,
    "keep_edges": True,
    "edge_dilate": 3,
    "apply_overlay": True,
}


N_POS = 150
N_NEG = 150

print("=== Generating ONEFOLDER dataset ===")
coco_synth, meta_synth = generate_onefolder(
    pool_df=full_pool,
    out_images_dir=OUT_IMAGES_DIR,
    n_pos=N_POS,
    n_neg=N_NEG,
    start_seed=3000,
    max_side=CFG["max_side"],
    steps=CFG["steps"],
    guidance_scale=CFG["guidance_scale"],
    controlnet_scale=CFG["controlnet_scale"],
    keep_pad=CFG["keep_pad"],
    sky_frac=CFG["sky_frac"],
    sea_from_frac=CFG["sea_from_frac"],
    keep_edges=CFG["keep_edges"],
    edge_dilate=CFG["edge_dilate"],
    apply_overlay=CFG["apply_overlay"],
)

# Save outputs
OUT_JSON = OUT_DIR / "instances_synth.json"
OUT_META = OUT_DIR / "meta_synth.csv"

with open(OUT_JSON, "w") as f:
    json.dump(coco_synth, f)

meta_synth.to_csv(OUT_META, index=False)

print("DONE.")
print("Images folder:", OUT_IMAGES_DIR)
print("COCO json:", OUT_JSON)
print("Meta csv:", OUT_META)



## Quick qualitative sanity check
- Randomly previews a few generated images to verify outputs visually.


In [ ]:
# -------------------------
#  Quick preview (random 6)
# -------------------------
import glob
all_imgs = sorted(glob.glob(str(OUT_IMAGES_DIR / "*.jpg")))
print("generated:", len(all_imgs))

if len(all_imgs) > 0:
    sample = random.sample(all_imgs, k=min(6, len(all_imgs)))
    for p in sample:
        show(Image.open(p).convert("RGB"), title=Path(p).name)